# install dependencies

In [ ]:
# !apt install default-jdk

In [1]:
%pip install -r requirements.txt
!python -m spacy download en_core_web_trf

  Using cached nmslib-2.1.2-cp311-cp311-linux_x86_64.whl
  Obtaining dependency information for pybind11 from https://files.pythonhosted.org/packages/06/55/9f73c32dda93fa4f539fafa268f9504e83c489f460c380371d94296126cd/pybind11-2.11.1-py3-none-any.whl.metadata
  Using cached pybind11-2.11.1-py3-none-any.whl.metadata (9.5 kB)
  Obtaining dependency information for pyserini==0.23.0 from https://files.pythonhosted.org/packages/bb/59/4a15b5da6f88bc1541598844d0a9982b8b04704a8083898c78a7eb8f1111/pyserini-0.23.0-py3-none-any.whl.metadata
  Using cached pyserini-0.23.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached faiss_cpu-1.7.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Obtaining dependency information for torch==2.1.1 from https://files.pythonhosted.org/packages/65/1f/de6bc1b246531db81d9a6afc491356b17d70e6c898e16f564970849f69b6/torch-2.1.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Using cached torch-2.1.1-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  

# download data and checkpoint

In [ ]:
!mkdir fever_wiki fever_wiki/corpora

# download data for retrieval
!curl -o ./fever_wiki/wiki-pages.zip https://fever.ai/download/fever/wiki-pages.zip
!unzip fever_wiki/wiki-pages.zip -d fever_wiki/

# download model checkpoints
!curl -o ./checkpoints/longformer_large_science.ckpt https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt &
!curl -o ./checkpoints/fever_sci.ckpt https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/fever_sci.ckpt &

In [ ]:
!python src/prepare_wiki.py

# index the corpus with pyserini

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input fever_wiki/corpora \
  --index indexes/fever_wiki \
  --generator DefaultLuceneDocumentGenerator \
  --threads 8 \
  --storePositions --storeDocvectors --storeRaw

# start streamlit

In [1]:
!streamlit run Entry.py \
    --server.port 8501 \
    --server.headless true \
    --client.toolbarMode minimal \
    --server.fileWatcherType none \
    -- \
    --checkpoint_path checkpoints/fever_sci.ckpt \
    --index_dir indexes/fever_wiki \
    --cuda \
    --batch_size 2


  You can now view your Streamlit app in your browser.

  Network URL: http://172.30.174.163:8501
  External URL: http://146.70.78.77:8501

/home/arch/Projects/college/Assignments/CS221_NLP/factcheck_en/dev_2/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/arch/Projects/college/Assignments/CS221_NLP/factcheck_en/dev_2/lib/python3.11/site-packages/pytorch_lightning/utilities/migration/migration.py:207: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
Lightning automatically upgraded your loaded checkpoint from v1.2.1 to v2.1.2. To apply the upgrade to

In [ ]:
%%html
<iframe src="http://172.24.112.107:8501" width="1200" height="1000"></iframe>